In [4]:
import  sqlalchemy as sqla 
import os

In [2]:
! pip install psycopg2-binary

# Leer Variables de ambiente

Estas variables de ambiente estan especificadas en el common.env, y la directiva para leerlas esta en el docker-compose.
Corresponden a los valores que elegimos al crear la base de datos.

In [5]:
POSTGRES_DB = os.getenv('POSTGRES_DB')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')

https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm

In [5]:
from sqlalchemy import Table, Column, Integer, String, MetaData

In [6]:
meta = MetaData()

Crear schema de la tabla

In [7]:
students = Table(
   'students', meta, 
   Column('cu', Integer, primary_key = True), 
   Column('nombre', String), 
   Column('apellido', String), 
)

Crear tabla en DB

In [8]:
meta.create_all(db)

2023-03-15 20:38:02,686 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-15 20:38:02,686 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-15 20:38:02,689 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-15 20:38:02,689 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-15 20:38:02,691 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-15 20:38:02,692 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-15 20:38:02,694 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-15 20:38:02,699 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

 Para ejecutar operaciones en la base de datos es necesario crear una coneccion

In [9]:
conn = db.connect()

## Ejecucion

### Insert

In [10]:
ins = students.insert()
print(ins)

INSERT INTO students (cu, nombre, apellido) VALUES (:cu, :nombre, :apellido)


In [11]:
ins = students.insert().values(cu=0, nombre = 'Hange', apellido='Zoe')
print(ins)
ins.compile().params

INSERT INTO students (cu, nombre, apellido) VALUES (:cu, :nombre, :apellido)


{'cu': 0, 'nombre': 'Hange', 'apellido': 'Zoe'}

Ejecutar la insercion

In [12]:
ins = students.insert()
ins = students.insert().values(cu=3, nombre = 'Hange', apellido='Zoe')
conn = db.connect()
result = conn.execute(ins)
print(result)

2023-03-15 20:38:02,746 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-15 20:38:02,747 INFO sqlalchemy.engine.Engine INSERT INTO students (cu, nombre, apellido) VALUES (%(cu)s, %(nombre)s, %(apellido)s)
2023-03-15 20:38:02,748 INFO sqlalchemy.engine.Engine [generated in 0.00213s] {'cu': 3, 'nombre': 'Hange', 'apellido': 'Zoe'}


Ejecutar Varias inserciones

In [13]:
conn.execute(students.insert(), [
   {'cu':'4', 'nombre' : 'Eren', 'apellido':'Jager'},
   {'cu':'5','nombre' : 'Armin', 'apellido':'Arlert'}
])

2023-03-15 20:38:02,755 INFO sqlalchemy.engine.Engine INSERT INTO students (cu, nombre, apellido) VALUES (%(cu__0)s, %(nombre__0)s, %(apellido__0)s), (%(cu__1)s, %(nombre__1)s, %(apellido__1)s)
2023-03-15 20:38:02,757 INFO sqlalchemy.engine.Engine [generated in 0.00011s (insertmanyvalues)] {'apellido__0': 'Jager', 'nombre__0': 'Eren', 'cu__0': '4', 'apellido__1': 'Arlert', 'nombre__1': 'Armin', 'cu__1': '5'}


### Select

In [14]:
s = students.select()
conn = db.connect()
result = conn.execute(s)

for row in result:
   print (row)

2023-03-15 20:38:02,767 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-15 20:38:02,769 INFO sqlalchemy.engine.Engine SELECT students.cu, students.nombre, students.apellido 
FROM students
2023-03-15 20:38:02,769 INFO sqlalchemy.engine.Engine [generated in 0.00253s] {}


# Crear Engine

In [6]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base
#from sqlalchemy.ext.declarative import declarative_base

In [7]:
# create a new engine
db_string = 'postgresql://{}:{}@{}:{}/{}'.format(POSTGRES_USER, POSTGRES_PASSWORD, POSTGRES_HOST, POSTGRES_PORT, POSTGRES_DB)
engine = create_engine(db_string, echo=True)

# Definir y Crear base

In [18]:
# create a new engine
db_string = 'postgresql://{}:{}@{}:{}/{}'.format(POSTGRES_USER, POSTGRES_PASSWORD, POSTGRES_HOST, POSTGRES_PORT, POSTGRES_DB)
engine = create_engine(db_string, echo=True)
# create a new base
Base = declarative_base()
# define the students table
class Student(Base):
    __tablename__ = 'students'
    cu = Column(Integer, primary_key=True)
    nombre = Column(String)
    apellido = Column(String)

# create the table
Base.metadata.create_all(engine)

2023-03-15 21:08:33,749 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-15 21:08:33,749 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-15 21:08:33,751 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-15 21:08:33,751 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-15 21:08:33,753 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-15 21:08:33,754 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-15 21:08:33,755 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-15 21:08:33,758 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

# Agregar Datos

In [26]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError
# Si la base esta definida en otro lado tienen que importarla
# from db_def import Student  



In [35]:
# create a metadata object and reflect the existing table
engine = create_engine(db_string, echo=False)
metadata = MetaData()
students_table = Table('students', metadata, autoload_with=engine)
Student.__table__ = students_table
Session = sessionmaker(bind=engine)
session = Session()

In [34]:
# Nuevos estudiandes
s1 = Student(cu=1, nombre = 'Hange', apellido='Zoe')
s2 =  Student(cu=2, nombre = 'Eren', apellido='Yeager')
s3 =  Student(cu=3, nombre = 'Armin', apellido='Arlert')


# add the students to the session
session.add_all([s1, s2, s3])

# commit the changes to the database
try:
    session.commit()
except IntegrityError:
    session.rollback()
    print('IntegrityError: Duplicate key value')

2023-03-15 21:15:02,432 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-15 21:15:02,436 INFO sqlalchemy.engine.Engine INSERT INTO students (cu, nombre, apellido) VALUES (%(cu__0)s, %(nombre__0)s, %(apellido__0)s), (%(cu__1)s, %(nombre__1)s, %(apellido__1)s), (%(cu__2)s, %(nombre__2)s, %(apellido__2)s)
2023-03-15 21:15:02,438 INFO sqlalchemy.engine.Engine [generated in 0.00016s (insertmanyvalues)] {'apellido__0': 'Zoe', 'nombre__0': 'Hange', 'cu__0': 1, 'apellido__1': 'Yeager', 'nombre__1': 'Eren', 'cu__1': 2, 'apellido__2': 'Arlert', 'nombre__2': 'Armin', 'cu__2': 3}
2023-03-15 21:15:02,442 INFO sqlalchemy.engine.Engine ROLLBACK
IntegrityError: Duplicate key value


# Ver tabla

In [40]:
from sqlalchemy import create_engine, MetaData, Table, select

# create a new engine
engine = create_engine(db_string, echo=False)

# create a metadata object and reflect the existing table
metadata = MetaData()
students_table = Table('students', metadata,  autoload_with=engine)

# create a select statement for the first 3 rows
select_stmt = select(students_table).limit(3)

# create a connection object and execute the select statement
conn = engine.connect()
result = conn.execute(select_stmt)

# print the rows
for row in result:
    print(row)

# close the connection
conn.close()

(1, 'Hange', 'Zoe')
(2, 'Eren', 'Yeager')
(3, 'Armin', 'Arlert')


# Ver Columnas

In [72]:
from sqlalchemy import create_engine, MetaData, Table

# create a new engine
engine = create_engine(db_string, echo=False)

# create a metadata object and reflect the existing table
metadata = MetaData()
students_table = Table('students', metadata, autoload_with=engine)

# print the list of column names
print(students_table.columns.keys())

['cu', 'nombre', 'apellido']


# Agregar Columna

Redefinir tabla

In [73]:
# create a new engine
db_string = 'postgresql://{}:{}@{}:{}/{}'.format(POSTGRES_USER, POSTGRES_PASSWORD, POSTGRES_HOST, POSTGRES_PORT, POSTGRES_DB)
engine = create_engine(db_string, echo=True)
# create a new base
Base = declarative_base()
# redefinir columna estudiantes
class Student(Base):
    __tablename__ = 'students'
    cu = Column(Integer, primary_key=True)
    nombre = Column(String)
    apellido = Column(String)
    grade = Column(Float)


In [76]:

# crear un objeto metadata y reflejar la tabla existente
metadata = MetaData()
students_table = Table('students', metadata, autoload_with=engine)

# agregar una nueva columna a la tabla
new_column = Column('grade', Float)
new_column.create(students_table.bind)# aplicar los cambios a la tabla
# guardar los cambios en la base de datos
metadata.create_all(engine)

2023-03-15 21:42:20,142 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-15 21:42:20,144 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_attribute.attname AS name, pg_catalog.format_type(pg_catalog.pg_attribute.atttypid, pg_catalog.pg_attribute.atttypmod) AS format_type, (SELECT pg_catalog.pg_get_expr(pg_catalog.pg_attrdef.adbin, pg_catalog.pg_attrdef.adrelid) AS pg_get_expr_1 
FROM pg_catalog.pg_attrdef 
WHERE pg_catalog.pg_attrdef.adrelid = pg_catalog.pg_attribute.attrelid AND pg_catalog.pg_attrdef.adnum = pg_catalog.pg_attribute.attnum AND pg_catalog.pg_attribute.atthasdef) AS "default", pg_catalog.pg_attribute.attnotnull AS not_null, pg_catalog.pg_class.relname AS table_name, pg_catalog.pg_description.description AS comment, pg_catalog.pg_attribute.attgenerated AS generated, (SELECT json_build_object(%(json_build_object_2)s, pg_catalog.pg_attribute.attidentity = %(attidentity_1)s, %(json_build_object_3)s, pg_catalog.pg_sequence.seqstart, %(json_build_object_4)s, pg_catalog

AttributeError: Neither 'Column' object nor 'Comparator' object has an attribute 'create'

In [65]:
# create a metadata object and reflect the existing table
engine = create_engine(db_string, echo=False)
metadata = MetaData()
students_table = Table('students', metadata, autoload_with=engine)
Student.__table__ = students_table
Session = sessionmaker(bind=engine)
session = Session()

In [66]:
# Nuevos estudiandes
s1 = Student(cu=4, nombre = 'Levi', apellido='Ackerman', grade=9.9)

# add the students to the session
session.add_all([s1])

# commit the changes to the database
try:
    session.commit()
except IntegrityError:
    session.rollback()
    print('IntegrityError: Duplicate key value')

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "grade" of relation "students" does not exist
LINE 1: INSERT INTO students (cu, nombre, apellido, grade) VALUES (4...
                                                    ^

[SQL: INSERT INTO students (cu, nombre, apellido, grade) VALUES (%(cu)s, %(nombre)s, %(apellido)s, %(grade)s)]
[parameters: {'cu': 4, 'nombre': 'Levi', 'apellido': 'Ackerman', 'grade': 9.9}]
(Background on this error at: https://sqlalche.me/e/20/f405)